Import all the relevant packages I will need

In [ ]:
import pandas as pd 
import pyodbc as p
from nltk.stem import *
import nltk
from nltk import word_tokenize

Connect to a specific Database. We will pass the serverName and database as parameters so users can define their own servers and databases. Makes it more flexible.

In [ ]:
def connectToDB(serverName, database):
    
    sql_conn = p.connect('DRIVER={ODBC Driver 17 for SQL Server}; SERVER=' + serverName + ';DATABASE=' + database + ';Trusted_Connection=yes;')       
    return sql_conn

The following code is a function that connects to the specified database (of users' choice) and then a retrieval query is executed and stored as a DataFrame. This is then written back to SSMS and the DataFrame is returned as an output. 

This code is only ever needed to when wanting to take a subset of a larger table and write to another place in the Database.

In [ ]:
#def writeDB(database):
#    conn = connectToDB('DESKTOP-7CLEOQN\SQLEXPRESS', database)
#    query =  "SELECT [CustomerID],[CustomerName],[CustomerReferenceNo],[AddressLine4] FROM dbo.Mock3Customer"
#    df = pd.read_sql(query, conn)
    
#    cursor = conn.cursor()
    
#    for index, row in df.iterrows():
#        cursor.execute("INSERT INTO [Mock3].[dbo].[test3]([CustomerID],[CustomerName],[CustomerReferenceNo],[PostCode]) values (?,?,?,?)",row['CustomerID'],row['CustomerName'],row['CustomerReferenceNo'],row['AddressLine4']) 
#        conn.commit()
#    cursor.close()
#    conn.close()
    
#    return df

The following function reads the desired query the user wishes to execute. For the purpose of my project, I will retrieve everything I wrote to the new test tables in the previous command for each of the databases and store them into a dataframe of its own. 

In [ ]:
def readDB(database):
    conn = connectToDB('DESKTOP-7CLEOQN\SQLEXPRESS', database)
    query =  "SELECT * FROM dbo.test1"
    df = pd.read_sql(query, conn)
    
    return df

In [ ]:
db1 = readDB('Mock')
print(db1)

We execute the following code to simply make sure we are retrieving everyting we expect. We can manually cross-validate by opening SSMS and seeing the same data points. Since this is a self-created table, we are only working with upto 15 data points at most.

In [ ]:
def readDB(database):
    conn = connectToDB('DESKTOP-7CLEOQN\SQLEXPRESS', database)
    query =  "SELECT * FROM dbo.test2"
    df = pd.read_sql(query, conn)
    
    return df

In [ ]:
db2 = readDB('Mock2')
print(db2)

In [ ]:
def readDB(database):
    conn = connectToDB('DESKTOP-7CLEOQN\SQLEXPRESS', database)
    query =  "SELECT * FROM dbo.test3"
    df = pd.read_sql(query, conn)
    
    return df

In [ ]:
db3 = readDB('Mock3')
print(db3)

We create a new column by merging two columns. In this case we are merging the CustomerName column with the PostCode since this will form our new key and aggregating them initially makes the tokenisation easier.

In [ ]:
db1['name_postcode'] = db1[['CustomerName', 'AddressLine4']].agg(' '.join, axis=1)

In [ ]:
print(db1)

In [ ]:
db2['name_postcode'] = db2[['CustomerName', 'AddressLine4']].agg(' '.join, axis=1)

In [ ]:
print(db2)

The following code does a Left join on the two DataFrames (db1 and db2). We see that as it stands, without employing our tool capabilities; there is only one match - the database identifies only one entity to exist the same across both tables. 
100% identical match.

In [ ]:
inner_join = pd.merge(db1, db2, on='CustomerName', how='left')

inner_join

The following function performs normalisation which will be used on our tokens. It takes any string and removes any punctuation within the string. It then turns every character left into lowercase so there is no variation between 'and' and 'And'.  

In [ ]:
def strip_punctuation(text):
    delim = dict.fromkeys(':,.?\n-&"?!')
    string = ' '
    sentence = [] 
  
    for word in text: 
        if word not in delim: 
            sentence.append(word) 
        else:
            sentence.append(' ')
            
    for i in sentence:
        string += i

    lower = string.lower() #make it all lowercase so we can avoid thinking about capitals (and/And)
    return lower

Perform tokenization (using NLTK library), remove stopwords etc.

In [ ]:
name_pc = db1['name_postcode']
tokens = []

for i in name_pc:
    no_punc = strip_punctuation(i)
    tokenize = word_tokenize(no_punc) #tokenization from the NLTK library
    
    #LIST of all company/customer name stopwords 
    stopwords = ['agency', 'gmbh', 'pa', 'assn', 'group', 'and', 'pc', 'hotel', 'pharmacy', 'assoc', 'hotels', 'plc', 'associates', 'inc', 'pllc', 'association', 'incorporated', 'restaurant', 'bank', 'international', 'sa', 'bv', 'intl', 'sales', 'co', 'limited', 'service', 'comp', 'llc', 'services', 'company', 'llp', 'store', 'corp', 'lp', 'svcs', 'corporation', 'ltd', 'travel', 'dmd', 'manufacturing', 'unlimited', 'enterprises', 'mfg']
    
    filtered_sentence = [w for w in tokenize if not w in stopwords] 
    tokens.append(filtered_sentence)

In [ ]:
keys = [''.join(i) for i in tokens]    

In [ ]:
db1['newKeys'] = keys

In [ ]:
print(db1) #This line of code is simply to show what we obtain

Before we can join the tables to see the effect of creating new keys, we will manually do the same for the entries in our second database table in this case db2.

In [ ]:
name_pc2 = db2['name_postcode']
tokens = []

for i in name_pc2:
    no_punc = strip_punctuation(i)
    tokenize = word_tokenize(no_punc) #tokenization from the NLTK library
    
    #LIST of all company/customer name stopwords 
    stopwords = ['agency', 'ltd', 'gmbh', 'pa', 'assn', 'group', 'and', 'pc', 'hotel', 'pharmacy', 'assoc', 'hotels', 'plc', 'associates', 'inc', 'pllc', 'association', 'incorporated', 'restaurant', 'bank', 'international', 'sa', 'bv', 'intl', 'sales', 'co', 'limited', 'service', 'comp', 'llc', 'services', 'company', 'llp', 'store', 'corp', 'lp', 'svcs', 'corporation', 'travel', 'dmd', 'manufacturing', 'unlimited', 'enterprises', 'mfg']
    
    filtered_sentence = [w for w in tokenize if not w in stopwords] 
    tokens.append(filtered_sentence)

In [ ]:
keys = [''.join(i) for i in tokens]    

Assign the newly created keys to a new column 'newKeys'. 

In [ ]:
db2['newKeys'] = keys

In [ ]:
print(db2)

Just to kind of show up until now the work that has been done we can see below: 

In [ ]:
name = db2['CustomerName']
postcode = db2['AddressLine4']         
print('Debtor Name:', '"', name[3], '"')
print('Debtor PostCode:', '"', postcode[3], '"')
print('Debtor Name + PostCode:', '"', name_pc2[3], '"')
print('Normalised Tokens:', '"', tokens[3], '"')

Now we will re run our code that does the left join on our two databases and see what difference it makes in terms of identifying the records in our first database to be anywhere else

In [ ]:
inner_join = pd.merge(db1, db2, on='newKeys', how='left')

inner_join

###### We can analyse there is an extra match since the three entities we have intentionally created to prove our concept is: 
- RuhiBegum&Co
- Sohiel And Family - Lion"s Den
- BMW Group

we have purposefully created a minor difference in the postcode for 'Sohiel And Family...'  and therefore we will explore this through fuzzy matching since it is not identical and has not been picked up.

This was a first simple approach.

# Fuzzy Partial Matching

Import the Fuzz packages that will allow us to do the fuzzy partial matching

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

Create a variable to compare the two columns hierarchically.


In [ ]:
compare = pd.MultiIndex.from_product([db1['newKeys'], db2['newKeys']]).to_series()

The following function creates our comparisons of newKeys for four different fuzz methods: 
- fuzz.ratio
- fuzz.partial_ratio
- fuzz.token_sort_ratio
- fuzz.token_set_ratio

In [ ]:
def metrics(tup):
#return pd.Series([fuzz.ratio(*tup), fuzz.token_sort_ratio(*tup)], ['ratio', 'token'])
    ratio = fuzz.ratio(*tup)
    partial = fuzz.partial_ratio(*tup)
    token = fuzz.token_sort_ratio(*tup)
    sett = fuzz.token_set_ratio(*tup)
    return pd.Series([ratio, partial, token, sett], ['ratio', 'partial_ratio', 'token_sort_ratio', 'token_set_ratio'])
     

In [ ]:
pd.options.display.max_rows = 100000

We display the comparisons in a table.


In [ ]:
table_of_comparisons = compare.apply(metrics)
table_of_comparisons

Since we do not need to acknowledge every single comparison, we set a threshold such that if two keys have a score greater or equal to 80 then it is one (same) entity. 
Our results of this are shown below. 

In [ ]:
table_of_comparisons[(table_of_comparisons['ratio'] >= 80) & (table_of_comparisons['partial_ratio'] >= 80) & (table_of_comparisons['token_sort_ratio'] >= 80) & (table_of_comparisons['token_set_ratio'] >= 80)]

In [ ]:
def fuzzy_partial(tup):
    token = fuzz.token_sort_ratio(*tup)
    return pd.Series([token], ['token_sort_ratio'])

In [ ]:
comp = compare.apply(fuzzy_partial)
comp[(comp['token_sort_ratio'] >= 80)]

Now we will test the effect of what we have so far against a third database

Similar to above, we write from the retrieved data of Mock3Customer into Mock3.dbo.test3 and read it in

In [ ]:
def readDB(database):
    conn = connectToDB('DESKTOP-7CLEOQN\SQLEXPRESS', database)
    query =  "SELECT * FROM dbo.test3"
    df = pd.read_sql(query, conn)
    
    return df

Simply execute a print statement just to verify we have loaded in the table correctly

In [ ]:
db3 = readDB('Mock3')
print(db3)

Again, Create a new column in the DataFrame which concatenates the CustomerName and the corresponding Postal Code


In [ ]:
db3['name_postcode'] = db3[['CustomerName', 'PostCode']].agg(' '.join, axis=1)

The following code will do a left join on our target database (Mock) against db3.
At this point, we expect to see zero joins since no Customer names are identical. 
However, we know through intentionally producing errors the entities that are same in both Database1 and Database3 are: 
- RuhiBegum&Co 
- Sohiel And Family - Lion's Den
- D.I. Cash And Carry 
- Bongos Dongos

In [ ]:
inner_join = pd.merge(db1, db3, on='CustomerName', how='left')

inner_join

Now we will carry out the preprocessing steps we have produced. 
The following code performs tokenisation, which has normalisation embedded on line 5


In [ ]:
name_pc = db3['name_postcode']
tokens = []

for i in name_pc:
    no_punc = strip_punctuation(i)
    tokenize = word_tokenize(no_punc) #tokenization from the NLTK library
    
    #LIST of all company/customer name stopwords 
    stopwords = ['agency', 'gmbh', 'pa', 'assn', 'group', 'and', 'pc', 'hotel', 'pharmacy', 'assoc', 'hotels', 'plc', 'associates', 'inc', 'pllc', 'association', 'incorporated', 'restaurant', 'bank', 'international', 'sa', 'bv', 'intl', 'sales', 'co', 'limited', 'service', 'comp', 'llc', 'services', 'company', 'llp', 'store', 'corp', 'lp', 'svcs', 'corporation', 'ltd', 'travel', 'dmd', 'manufacturing', 'unlimited', 'enterprises', 'mfg']
    
    filtered_sentence = [w for w in tokenize if not w in stopwords] 
    tokens.append(filtered_sentence)
    

From the pre-processing steps above, we produce our new keys


In [ ]:
keys = [''.join(i) for i in tokens]    
db3['newKeys'] = keys

Let us now perform an inner join simply after having produced newKeys. There is no partial matching at this point yet.
For this reason, we only expect to see 
- D.I. Cash And Carry

join since the keys are now identical 

In [ ]:
inner_join = pd.merge(db1, db3, on='newKeys', how='left')

inner_join

Now we will use our methods created for Fuzzy Partial Matching. 

Remember we apply a threshold of >=80 since this accommodates greater spelling flaws etc.


In [ ]:
compare = pd.MultiIndex.from_product([db1['newKeys'], db3['newKeys']]).to_series()

In [ ]:
pd.options.display.max_rows = 100000
table_of_comparisons = compare.apply(metrics)
table_of_comparisons

In [ ]:
comp = compare.apply(fuzzy_partial)
comp[(comp['token_sort_ratio'] >= 80)]

Let us validate this one more time by retrieving entities that exist in Database 2 and Database 3. 
I jave purposely created more entities that should be retrieved. 
We should retrieve:
- RuhiBegum&Co 
- Sohiel And Family - Lion's Den
- Barclays Bank - Birmingham 
- Rojers Savouries
- Sunpride
- Lidl Cash And Carry 

In [ ]:
compare = pd.MultiIndex.from_product([db2['newKeys'], db3['newKeys']]).to_series()

In [ ]:
pd.options.display.max_rows = 100000
table_of_comparisons = compare.apply(metrics)

In [ ]:
comp = compare.apply(fuzzy_partial)
comp[(comp['token_sort_ratio'] >= 80)]

We can see again, fuzzy partial matching accomodates for spelling errors, were characters are physically missing etc. These are features Fuzz.token_sort_ratio deals with better than partial_ratio but further research into different methods is required.


For future consideration the necessity of address false positives/false negatives remains essential.
For example, we can see cashcarry make bulk of the sequence for the key 'lidlcashcarrysy128gj' therefore, another entry also retains this phrase and therefore identified as similar. 

In [ ]:
comp[(comp['token_sort_ratio'] >= 50)]

# Thank You for listening